In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, r2_score

In [44]:
train_clin_df = pd.read_csv('../data/raw/train_clinical_data.csv')
train_prot_df = pd.read_csv('../data/raw/train_proteins.csv')
train_pep_df = pd.read_csv('../data/raw/train_peptides.csv')

test_df = pd.read_csv('../data/raw/test.csv')
test_prot_df = pd.read_csv('../data/raw/test_proteins.csv')
test_pep_df = pd.read_csv('../data/raw/test_peptides.csv')

sample_submission = pd.read_csv('../data/raw/sample_submission.csv')

In [45]:
sample_submission.head(10)

,prediction_id,rating,group_key
0,3342_0_updrs_1_plus_0_months,0,0
1,3342_0_updrs_1_plus_6_months,0,0
2,3342_0_updrs_1_plus_12_months,0,0
3,3342_0_updrs_1_plus_24_months,0,0
4,3342_0_updrs_2_plus_0_months,0,0
5,3342_0_updrs_2_plus_6_months,0,0
6,3342_0_updrs_2_plus_12_months,0,0
7,3342_0_updrs_2_plus_24_months,0,0
8,3342_0_updrs_3_plus_0_months,0,0
9,3342_0_updrs_3_plus_6_months,0,0


In [46]:
sample_submission.tail()

,prediction_id,rating,group_key
59,50423_6_updrs_3_plus_24_months,0,6
60,50423_6_updrs_4_plus_0_months,0,6
61,50423_6_updrs_4_plus_6_months,0,6
62,50423_6_updrs_4_plus_12_months,0,6
63,50423_6_updrs_4_plus_24_months,0,6


In [47]:
sample_submission.sort_values(by='prediction_id')[:20]

,prediction_id,rating,group_key
0,3342_0_updrs_1_plus_0_months,0,0
2,3342_0_updrs_1_plus_12_months,0,0
3,3342_0_updrs_1_plus_24_months,0,0
1,3342_0_updrs_1_plus_6_months,0,0
4,3342_0_updrs_2_plus_0_months,0,0
6,3342_0_updrs_2_plus_12_months,0,0
7,3342_0_updrs_2_plus_24_months,0,0
5,3342_0_updrs_2_plus_6_months,0,0
8,3342_0_updrs_3_plus_0_months,0,0
10,3342_0_updrs_3_plus_12_months,0,0


In [48]:
test_df.head()

,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0


In [49]:
test_df[test_df['patient_id'] == 3342]

,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
8,3342_6,6,3342,updrs_1,3342_6_updrs_1,6
9,3342_6,6,3342,updrs_2,3342_6_updrs_2,6
10,3342_6,6,3342,updrs_3,3342_6_updrs_3,6
11,3342_6,6,3342,updrs_4,3342_6_updrs_4,6


In [50]:
train_clin_df.head(20)

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
5,55_18,55,18,7.0,13.0,38.0,0.0,On
6,55_24,55,24,16.0,9.0,49.0,0.0,On
7,55_30,55,30,14.0,13.0,49.0,0.0,On
8,55_36,55,36,17.0,18.0,51.0,0.0,On
9,55_42,55,42,12.0,20.0,41.0,0.0,On


In [51]:
train_clin_df['upd23b_clinical_state_on_medication'].value_counts()

On     775
Off    513
Name: upd23b_clinical_state_on_medication, dtype: int64

In [52]:
train_pep_df.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [53]:
train_prot_df.head()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [54]:
# create a column with the UniProt and Peptide name combined
train_pep_df['peptide_uniprot'] = train_pep_df['Peptide'] + '_'+ train_pep_df['UniProt']
train_pep_df.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,peptide_uniprot
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3,NEQEQPLGQWHLS_O00391
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0,GNPEPTFSWTK_O00533
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0,IEIPSSVQQVPTIIK_O00533
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK_O00533
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7,SMEQNGPGLEYR_O00533


In [55]:
# create a table with the visit_id as the index and the proteins or peptides as the feature and the abundance as the values
train_prot_pivot = train_prot_df.pivot(index='visit_id', values='NPX', columns='UniProt')
train_pep_pivot = train_pep_df.pivot(index='visit_id', values='PeptideAbundance', columns='peptide_uniprot')

In [56]:
# combine the two tables on the visit_id
full_prot_train_df = train_prot_pivot.join(train_pep_pivot)
full_prot_train_df.head()

,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,...,YSLTYIYTGLSK_P25311,YTTEIIK_P00736,YVGGQEHFAHLLILR_P02763,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR_P00738,YVMLPVADQDQC(UniMod_4)IR_P00738,YVNKEIQNAVNGVK_P10909,YWGVASFLQK_P02753,YYC(UniMod_4)FQGNQFLR_P02790,YYTYLIMNK_P01024,YYWGGQYTWDMAK_P02675
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,129048.0,...,202274.0,NaN,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,NaN,7207.30
10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,108114.0,...,201009.0,NaN,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,163776.0,...,220728.0,NaN,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
10138_12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,56725.0,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
10138_24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,85767.1,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,NaN,56977.6,4903.09


In [57]:
# fill nan with 0 for this first round
full_prot_train_df = full_prot_train_df.fillna(0)

In [58]:
train_clin_df.head(10)

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
5,55_18,55,18,7.0,13.0,38.0,0.0,On
6,55_24,55,24,16.0,9.0,49.0,0.0,On
7,55_30,55,30,14.0,13.0,49.0,0.0,On
8,55_36,55,36,17.0,18.0,51.0,0.0,On
9,55_42,55,42,12.0,20.0,41.0,0.0,On


In [59]:
train_clin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2615 entries, 0 to 2614
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   visit_id                             2615 non-null   object 
 1   patient_id                           2615 non-null   int64  
 2   visit_month                          2615 non-null   int64  
 3   updrs_1                              2614 non-null   float64
 4   updrs_2                              2613 non-null   float64
 5   updrs_3                              2590 non-null   float64
 6   updrs_4                              1577 non-null   float64
 7   upd23b_clinical_state_on_medication  1288 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 163.6+ KB


#### For updrs_1, updrs_2, and updrs_3 all null rows can be dropped since they are a small amount totaling about 1% of the rows.

#### Also the first round of predictions will be on only updrs_1, updrs_2, and updrs_3 only, so updrs_4 can be dropped completely

In [60]:
train_clin_df = train_clin_df.iloc[:, :6]
train_clin_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3
0,55_0,55,0,10.0,6.0,15.0
1,55_3,55,3,10.0,7.0,25.0
2,55_6,55,6,8.0,10.0,34.0
3,55_9,55,9,8.0,9.0,30.0
4,55_12,55,12,10.0,10.0,41.0


In [61]:
train_clin_df = train_clin_df.dropna()
train_clin_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2588 entries, 0 to 2614
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   visit_id     2588 non-null   object 
 1   patient_id   2588 non-null   int64  
 2   visit_month  2588 non-null   int64  
 3   updrs_1      2588 non-null   float64
 4   updrs_2      2588 non-null   float64
 5   updrs_3      2588 non-null   float64
dtypes: float64(3), int64(2), object(1)
memory usage: 141.5+ KB


In [62]:
full_train_df = train_clin_df.merge(full_prot_train_df, how='right', left_on='visit_id', right_on='visit_id')
full_train_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,O00391,O00533,O00584,O14498,...,YSLTYIYTGLSK_P25311,YTTEIIK_P00736,YVGGQEHFAHLLILR_P02763,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR_P00738,YVMLPVADQDQC(UniMod_4)IR_P00738,YVNKEIQNAVNGVK_P10909,YWGVASFLQK_P02753,YYC(UniMod_4)FQGNQFLR_P02790,YYTYLIMNK_P01024,YYWGGQYTWDMAK_P02675
0,10053_0,10053.0,0.0,3.0,0.0,13.0,9104.27,402321.0,0.00,0.0,...,202274.0,0.00,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,0.0,7207.30
1,10053_12,10053.0,12.0,4.0,2.0,8.0,10464.20,435586.0,0.00,0.0,...,201009.0,0.00,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
2,10053_18,10053.0,18.0,2.0,2.0,0.0,13235.70,507386.0,7126.96,24525.7,...,220728.0,0.00,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
3,10138_12,10138.0,12.0,3.0,6.0,31.0,12600.20,494581.0,9165.06,27193.5,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
4,10138_24,10138.0,24.0,4.0,7.0,19.0,12003.20,522138.0,4498.51,17189.8,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,0.0,56977.6,4903.09


### Use the right join to eliminate any clinic visits that did not have the protein and peptides taken

### Try Random Forrest Regressor to start on updrs_1 only

target column will be updrs_1 and training data will be all the peptide and protein data.

visit_month, patient_id, and visit_month will all be excluded

In [63]:
# view the first target value counts
full_train_df['updrs_1'].value_counts()

1.0     117
2.0     105
5.0      89
6.0      88
4.0      78
3.0      77
7.0      67
8.0      64
0.0      61
9.0      57
11.0     42
10.0     41
12.0     32
14.0     24
13.0     21
16.0     17
15.0     15
20.0     13
17.0     12
18.0     11
19.0      9
25.0      5
21.0      3
23.0      2
26.0      2
22.0      2
24.0      1
27.0      1
28.0      1
33.0      1
Name: updrs_1, dtype: int64

#### Perform KFold Cross Validation on the Data for the Model

In [64]:
# randomize the rows
full_train_df = full_train_df.sample(frac=1).reset_index(drop=True)
full_train_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,O00391,O00533,O00584,O14498,...,YSLTYIYTGLSK_P25311,YTTEIIK_P00736,YVGGQEHFAHLLILR_P02763,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR_P00738,YVMLPVADQDQC(UniMod_4)IR_P00738,YVNKEIQNAVNGVK_P10909,YWGVASFLQK_P02753,YYC(UniMod_4)FQGNQFLR_P02790,YYTYLIMNK_P01024,YYWGGQYTWDMAK_P02675
0,29313_6,29313.0,6.0,6.0,1.0,17.0,12091.20,290090.0,25652.60,31288.9,...,209437.0,9781.47,2892880.0,118819.0,1684560.0,108770.0,119872.0,613070.0,49975.7,24248.0
1,49239_36,49239.0,36.0,11.0,11.0,26.0,13589.70,1064420.0,63575.50,46719.6,...,166223.0,18378.40,2676020.0,60879.9,295033.0,116742.0,114126.0,339034.0,69195.0,0.0
2,42385_24,42385.0,24.0,3.0,0.0,0.0,8769.71,591324.0,9298.51,27472.4,...,184732.0,7151.46,2621040.0,19424.4,125889.0,108429.0,112764.0,372088.0,43826.1,20697.1
3,47881_0,47881.0,0.0,1.0,2.0,14.0,11369.40,233674.0,23339.50,19255.9,...,280227.0,4318.67,6144270.0,73488.0,845429.0,64338.3,123588.0,441960.0,36632.6,23155.5
4,26210_24,26210.0,24.0,5.0,4.0,5.0,10723.20,529157.0,28389.00,23860.8,...,227976.0,8347.46,2581610.0,0.0,168330.0,157205.0,134560.0,357203.0,38156.7,14699.0


In [65]:
full_train_df.shape

(1113, 1201)

In [66]:
full_train_df['updrs_1'].isna().sum()

55

In [67]:
# calculate the number of bins by Sturge's rule
num_bins = int(np.floor(1 + np.log2(len(full_train_df))))
print(num_bins)

11


In [68]:
# bin the target updrs_1
full_train_df.loc[:, "bins"] = pd.cut(full_train_df['updrs_1'], bins=num_bins, labels=False)
full_train_df['bins'].head()

0    1.0
1    3.0
2    0.0
3    0.0
4    1.0
Name: bins, dtype: float64

In [69]:
full_train_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,O00391,O00533,O00584,O14498,...,YTTEIIK_P00736,YVGGQEHFAHLLILR_P02763,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR_P00738,YVMLPVADQDQC(UniMod_4)IR_P00738,YVNKEIQNAVNGVK_P10909,YWGVASFLQK_P02753,YYC(UniMod_4)FQGNQFLR_P02790,YYTYLIMNK_P01024,YYWGGQYTWDMAK_P02675,bins
0,29313_6,29313.0,6.0,6.0,1.0,17.0,12091.20,290090.0,25652.60,31288.9,...,9781.47,2892880.0,118819.0,1684560.0,108770.0,119872.0,613070.0,49975.7,24248.0,1.0
1,49239_36,49239.0,36.0,11.0,11.0,26.0,13589.70,1064420.0,63575.50,46719.6,...,18378.40,2676020.0,60879.9,295033.0,116742.0,114126.0,339034.0,69195.0,0.0,3.0
2,42385_24,42385.0,24.0,3.0,0.0,0.0,8769.71,591324.0,9298.51,27472.4,...,7151.46,2621040.0,19424.4,125889.0,108429.0,112764.0,372088.0,43826.1,20697.1,0.0
3,47881_0,47881.0,0.0,1.0,2.0,14.0,11369.40,233674.0,23339.50,19255.9,...,4318.67,6144270.0,73488.0,845429.0,64338.3,123588.0,441960.0,36632.6,23155.5,0.0
4,26210_24,26210.0,24.0,5.0,4.0,5.0,10723.20,529157.0,28389.00,23860.8,...,8347.46,2581610.0,0.0,168330.0,157205.0,134560.0,357203.0,38156.7,14699.0,1.0


In [70]:
# drop the NaN rows
full_train_df = full_train_df.dropna().reset_index(drop=True)
full_train_df.shape

(1058, 1202)

In [71]:
# initiate the kfold class from sklearn
kf = StratifiedKFold(n_splits=5)

In [72]:
# create a kfold column
full_train_df['kfold'] = -1

# fill the kfold column
for f, (t_, v_) in enumerate(kf.split(X=full_train_df, y=full_train_df['bins'].values)):
    full_train_df.loc[v_, 'kfold'] = f

C:\Users\Dagart\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


In [73]:
# drop the bins column
full_train_df = full_train_df.drop('bins', axis=1)


In [74]:
full_train_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,O00391,O00533,O00584,O14498,...,YTTEIIK_P00736,YVGGQEHFAHLLILR_P02763,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR_P00738,YVMLPVADQDQC(UniMod_4)IR_P00738,YVNKEIQNAVNGVK_P10909,YWGVASFLQK_P02753,YYC(UniMod_4)FQGNQFLR_P02790,YYTYLIMNK_P01024,YYWGGQYTWDMAK_P02675,kfold
0,29313_6,29313.0,6.0,6.0,1.0,17.0,12091.20,290090.0,25652.60,31288.9,...,9781.47,2892880.0,118819.0,1684560.0,108770.0,119872.0,613070.0,49975.7,24248.0,0
1,49239_36,49239.0,36.0,11.0,11.0,26.0,13589.70,1064420.0,63575.50,46719.6,...,18378.40,2676020.0,60879.9,295033.0,116742.0,114126.0,339034.0,69195.0,0.0,0
2,42385_24,42385.0,24.0,3.0,0.0,0.0,8769.71,591324.0,9298.51,27472.4,...,7151.46,2621040.0,19424.4,125889.0,108429.0,112764.0,372088.0,43826.1,20697.1,0
3,47881_0,47881.0,0.0,1.0,2.0,14.0,11369.40,233674.0,23339.50,19255.9,...,4318.67,6144270.0,73488.0,845429.0,64338.3,123588.0,441960.0,36632.6,23155.5,0
4,26210_24,26210.0,24.0,5.0,4.0,5.0,10723.20,529157.0,28389.00,23860.8,...,8347.46,2581610.0,0.0,168330.0,157205.0,134560.0,357203.0,38156.7,14699.0,0


In [75]:
full_train_df.shape

(1058, 1202)

In [76]:
train_csv_df = full_train_df.drop(columns=['kfold'])
train_csv_df.to_csv('../data/processed/train_data.csv', index=False)

In [91]:
def calculate_smape(y_true, y_pred):

    return round(np.mean(np.abs(y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2)) * 100, 2)

In [92]:
def rf_cv_model(df, target, first_month=None):
    
    final_smape = list()
    
    for fold in df['kfold'].unique():
        print(f'Target: {target}')
        print(f'Fold: {fold}')
        kfold = fold
        
        if first_month:
            val_df = df[(df['kfold'] == kfold)&(df['visit_month'] == 0)].reset_index(drop=True)
        else:
            val_df = df[df['kfold'] == kfold].reset_index(drop=True)
        train_df = df[df['kfold'] != kfold].reset_index(drop=True)
        
        updr_list = ['updrs_1', 'updrs_2', 'updrs_3']
        drop_list = ['visit_id', 'patient_id'] + updr_list
    
        y_train = train_df[[target]]
        X_train = train_df.drop(drop_list, axis=1)
        y_val = val_df[[target]]
        X_val = val_df.drop(drop_list, axis=1)

        rf = RandomForestRegressor()

        rf.fit(X_train.values, y_train.values.ravel()) 

        val_preds = rf.predict(X_val.values)
        smape = calculate_smape(y_val.values.ravel(), val_preds)
        r2 = r2_score(y_val.values.ravel(), val_preds)
        
        print(f'SMAPE: {smape}')
        print(f'R-squared: {r2}\n')
        
        final_smape.append(smape)
        
    print('Average SMAPE:', np.mean(final_smape))

In [93]:
for updr in ['updrs_1', 'updrs_2', 'updrs_3']:
    rf_cv_model(full_train_df, updr, first_month=True)

Target: updrs_1
Fold: 0
SMAPE: 59.03
R-squared: 0.1483399319583245

Target: updrs_1
Fold: 1
SMAPE: 85.0
R-squared: 0.17735647604450722

Target: updrs_1
Fold: 2
SMAPE: 54.54
R-squared: 0.3247943282381335

Target: updrs_1
Fold: 3
SMAPE: 81.2
R-squared: -0.17919251706873007

Target: updrs_1
Fold: 4
SMAPE: 80.8
R-squared: -0.1427500120094154

Average SMAPE: 72.114
Target: updrs_2
Fold: 0
SMAPE: 107.93
R-squared: 0.012378721487366517

Target: updrs_2
Fold: 1
SMAPE: 100.39
R-squared: -0.3848605529536182

Target: updrs_2
Fold: 2
SMAPE: 83.06
R-squared: 0.3608483970577159

Target: updrs_2
Fold: 3
SMAPE: 90.59
R-squared: -0.31449615177183476

Target: updrs_2
Fold: 4
SMAPE: 118.49
R-squared: -0.3898578699253976

Average SMAPE: 100.09200000000001
Target: updrs_3
Fold: 0
SMAPE: 92.36
R-squared: 0.06780675425990546

Target: updrs_3
Fold: 1
SMAPE: 78.5
R-squared: 0.04967487358536016

Target: updrs_3
Fold: 2
SMAPE: 75.08
R-squared: 0.34573776876224116

Target: updrs_3
Fold: 3
SMAPE: 76.32
R-squared: 

### Try XGBoost Regressor

In [96]:

import xgboost

def xgboost_cv_model(df, target, first_month=None):

    final_smape = list()

    for fold in df['kfold'].unique():
        print(f'Target: {target}')
        print(f'Fold: {fold}')
        kfold = fold
        
        if first_month:
            val_df = df[(df['kfold'] == kfold)&(df['visit_month'] == 0)].reset_index(drop=True)
        else:
            val_df = df[df['kfold'] == kfold].reset_index(drop=True)
        train_df = df[df['kfold'] != kfold].reset_index(drop=True)
        
        updr_list = ['updrs_1', 'updrs_2', 'updrs_3']
        drop_list = ['visit_id', 'patient_id'] + updr_list
    
        y_train = train_df[[target]]
        X_train = train_df.drop(drop_list, axis=1)
        y_val = val_df[[target]]
        X_val = val_df.drop(drop_list, axis=1)

        xgb = xgboost.XGBRegressor()

        xgb.fit(X_train.values, y_train.values.ravel()) 

        val_preds = xgb.predict(X_val.values)
        smape = calculate_smape(y_val.values.ravel(), val_preds)
    
        print(f'SMAPE: {smape}\n')
        
        final_smape.append(smape)
        
    print('Average SMAPE:', np.mean(final_smape))

In [97]:
for updr in ['updrs_1', 'updrs_2', 'updrs_3']:
    xgboost_cv_model(full_train_df, updr, first_month=True)

Target: updrs_1
Fold: 0
SMAPE: 62.39

Target: updrs_1
Fold: 1
SMAPE: 84.8

Target: updrs_1
Fold: 2
SMAPE: 51.77

Target: updrs_1
Fold: 3
SMAPE: 80.59

Target: updrs_1
Fold: 4
SMAPE: 79.56

Average SMAPE: 71.822
Target: updrs_2
Fold: 0
SMAPE: 105.77

Target: updrs_2
Fold: 1
SMAPE: 102.64

Target: updrs_2
Fold: 2
SMAPE: 88.58

Target: updrs_2
Fold: 3
SMAPE: 93.43

Target: updrs_2
Fold: 4
SMAPE: 123.02

Average SMAPE: 102.68800000000002
Target: updrs_3
Fold: 0
SMAPE: 89.73

Target: updrs_3
Fold: 1
SMAPE: 82.22

Target: updrs_3
Fold: 2
SMAPE: 59.52

Target: updrs_3
Fold: 3
SMAPE: 72.96

Target: updrs_3
Fold: 4
SMAPE: 98.3

Average SMAPE: 80.546
